### 1. 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2. selenium 설치 & 내 드라이브에 chromedriver 설치

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' # (최초 1회)
!pip install chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 23.6 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,604 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [75.2 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main a

In [ ]:
# selenium 설치 확인
!python --version

import selenium
print(selenium.__version__)

Python 3.11.12
4.31.0


### 3. 실행 준비

#### 3.1 라이브러리 임포트

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import chromedriver_autoinstaller  # setup chrome options

In [ ]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

#### 3.1 chrome_options 설정

In [ ]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off : cloab은 새창을 지원하지않기 때문에 창 없는 모드
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL


#### 3.3 자동 스크롤 함수

In [ ]:
def selenium_scroll_option():
  SCROLL_PAUSE_SEC = 3

  # 스크롤 높이 가져옴
  last_height = driver.execute_script("return document.body.scrollHeight")

  while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 10초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break
    last_height = new_height

### 4. 이미지 크롤링

In [ ]:
url = "https://www.google.com/"  # set up the webdriver

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# driver 작동 테스트
driver.get(url)
driver.implicitly_wait(3) # element가 로드될 때까지 지정한 시간만큼 대기할 수 있도록 설정
# driver.get_screenshot_as_file('google_screen.png')

driver.close()

print(driver)

<selenium.webdriver.chrome.webdriver.WebDriver (session="84c8197c8c3695ca1ce3f0f4f5d5c1c2")>


In [ ]:
courbet = "/content/drive/MyDrive/Colab Notebooks/"

In [ ]:
# 키워드 검색하기

base_url = "https://www.google.co.kr/imghp?hl=ko" # 구글 이미지 검색
keyword=input("검색할 키워드를 입력 : ")
# print(type(a))

image_name = input("저장할 이미지 이름 : ")

검색할 키워드를 입력 : 두산베어스
저장할 이미지 이름 : 두산베어스


In [ ]:
driver = webdriver.Chrome(options=chrome_options)

driver.get(base_url)

# driver.find_element("xpath",'//*[@id="APjFqb"]') # 정교한 선택이 필요할 때
# browser = driver.find_element(By.ID, "APjFqb") # 가장 빠르고 명확
browser =  driver.find_element(By.NAME, "q") # 구글 검색창에서 매우 안정적

browser.clear()
browser.send_keys(keyword)
browser.send_keys(Keys.RETURN)

In [ ]:
selenium_scroll_option() # 스크롤하여 이미지를 많이 확보

'''이미지 src요소를 리스트업해서 이미지 url 저장'''

images = driver.find_elements(By.CSS_SELECTOR,".rg_i.Q4LuWd")

images_url = []

for i in images:
    src = i.get_attribute('src') or i.get_attribute('data-src')
    if src:
        images_url.append(src)

print(f"수집한 이미지 URL 개수: {len(images_url)}")

driver.close()

수집한 이미지 URL 개수: 0


In [ ]:
# 겹치는 이미지 url 제거

print("전체 다운로드한 이미지 개수: {}\n동일한 이미지를 제거한 이미지 개수: {}".format(len(images_url), len(pd.DataFrame(images_url)[0].unique())))
images_url=pd.DataFrame(images_url)[0].unique()

if image_name == 'courbet' :
    for t, url in enumerate(images_url, 0):
        urlretrieve(url, courbet + image_name + '_' + str(t) + '.jpg')
    # driver.close()

KeyError: 0

In [ ]:
# 자원회수 - 코드 실행 마지막에 quit driver
driver.quit()